# Diseño y Vlsm 

# 📘 Documentación Técnica Final - Proyecto Carnaval Conectado (Grupo 10)

## 1. Criterios de Diseño
* **Bloque Asignado:** `10.2.64.0 /18`.
* **Estrategia:** Base 200 hosts por subred (Máscara **/24** estandarizada).
* **VLANs:** Esquema unificado para todas las sedes.

---

## 1.2 Justificación de Capacidad y Asignación (Base 200)

**A. Cálculo del Requerimiento Mínimo**
Se calcula la capacidad mínima requerida por sede utilizando la fórmula de base reducida:

$$Requerimiento = Base (200) + (Grupo (10) \times N_{vlan})$$

Considerando un escenario promedio de 5 VLANs operativas por sede (Admin, Logística, Prensa, Seguridad, Nativa):
$$200 + (10 \times 5) = \mathbf{250 \text{ hosts por sede}}$$

**B. Estrategia de Asignación (Over-provisioning)**
Aunque el requerimiento mínimo es de 250 hosts *totales* por sede, el diseño opta por asignar un bloque **Clase C (/24)** completo a *cada* VLAN individual.

* **Capacidad de Diseño:** 5 VLANs $\times$ 254 hosts/VLAN = **1,270 hosts totales**.
* **Holgura:** El diseño supera el requerimiento mínimo en un **400%**.

**C. Justificación Técnica de la Estrategia**
Se eligió esta asignación "generosa" basándose en tres pilares:

1.  **Disponibilidad de Recursos:** El grupo dispone de un bloque masivo `/18` (16,384 direcciones). Utilizar bloques `/24` apenas consume el 8% del espacio total disponible, por lo que no es necesario "ahorrar" direcciones.
2.  **Reducción de Errores Humanos (Simplicidad):** Al estandarizar todas las máscaras de subred a **`/24` (255.255.255.0)**, se elimina la complejidad de calcular máscaras variables (VLSM estricto) en los equipos de soporte y configuración, reduciendo drásticamente la probabilidad de fallos en la implementación.
3.  **Escalabilidad:** Esta arquitectura permite que cualquier departamento (ej. Prensa) crezca hasta 254 dispositivos sin necesidad de reconfigurar routers ni cambiar máscaras, garantizando la longevidad del diseño.


## 2. Esquema de VLANs (Actualizado)
Se añade la VLAN de Control y se renombra la de Monitoreo.

| VLAN ID | Nombre | Función | Segmento de Red (Patrón) |
| :--- | :--- | :--- | :--- |
| **10** | **ADMIN** | Administrativa y Coordinación | `10.2.X.0` |
| **20** | **LOGISTICA** | Operaciones y Producción | `10.2.(X+1).0` |
| **30** | **PRENSA** | Medios y Wi-Fi Invitados | `10.2.(X+2).0` |
| **40** | **MONITOREO** | Cámaras y Sensores (Antes Seg_IoT) | `10.2.(X+3).0` |
| **50** | **CONTROL** | Servidores, Datacenter y Sistemas Críticos | *IPs Específicas por sede* |
| **99** | **NATIVA** | Gestión de equipos (SSH) | `10.2.(X+4).0` |

---

## 3. Direccionamiento IP por Sede (LAN)

### 📍 Sede RÍO DE JANEIRO (Sede Central)
**Bloque Base:** `10.2.64.0`

| VLAN | Nombre | Dirección de Red | Gateway (Router) | Máscara |
| :--- | :--- | :--- | :--- | :--- |
| **10** | ADMIN | `10.2.64.0` | **`10.2.64.1`** | `/24` |
| **20** | LOGISTICA | `10.2.65.0` | **`10.2.65.1`** | `/24` |
| **30** | PRENSA | `10.2.66.0` | **`10.2.66.1`** | `/24` |
| **40** | MONITOREO | `10.2.67.0` | **`10.2.67.1`** | `/24` |
| **99** | NATIVA | `10.2.68.0` | **`10.2.68.1`** | `/24` |
| **50** | CONTROL | `10.2.70.0` | **`10.2.70.1`** | `/24` |

*(Nota: La VLAN Control aquí aloja también al Datacenter. Se saltó la .69 para dejar espacio).*

### 📍 Sede SÃO PAULO (Producción)
**Bloque Base:** `10.2.72.0`
*(Se movió desde la .70 para dejarle esa IP a Río).*

| VLAN | Nombre | Dirección de Red | Gateway (Router) | Máscara |
| :--- | :--- | :--- | :--- | :--- |
| **10** | ADMIN | `10.2.72.0` | **`10.2.72.1`** | `/24` |
| **20** | LOGISTICA | `10.2.73.0` | **`10.2.73.1`** | `/24` |
| **30** | PRENSA | `10.2.74.0` | **`10.2.74.1`** | `/24` |
| **40** | MONITOREO | `10.2.75.0` | **`10.2.75.1`** | `/24` |
| **99** | NATIVA | `10.2.76.0` | **`10.2.76.1`** | `/24` |
| **50** | CONTROL | `10.2.77.0` | **`10.2.77.1`** | `/24` |

### 📍 Sede SALVADOR (Operativa)
**Bloque Base:** `10.2.80.0`

| VLAN | Nombre | Dirección de Red | Gateway (Router) | Máscara |
| :--- | :--- | :--- | :--- | :--- |
| **10** | ADMIN | `10.2.80.0` | **`10.2.80.1`** | `/24` |
| **20** | LOGISTICA | `10.2.81.0` | **`10.2.81.1`** | `/24` |
| **30** | PRENSA | `10.2.82.0` | **`10.2.82.1`** | `/24` |
| **40** | MONITOREO | `10.2.83.0` | **`10.2.83.1`** | `/24` |
| **99** | NATIVA | `10.2.84.0` | **`10.2.84.1`** | `/24` |
| **50** | CONTROL | `10.2.85.0` | **`10.2.85.1`** | `/24` |

### 📍 Sede RECIFE (Soporte)
**Bloque Base:** `10.2.90.0`

| VLAN | Nombre | Dirección de Red | Gateway (Router) | Máscara |
| :--- | :--- | :--- | :--- | :--- |
| **10** | ADMIN | `10.2.90.0` | **`10.2.90.1`** | `/24` |
| **20** | LOGISTICA | `10.2.91.0` | **`10.2.91.1`** | `/24` |
| **30** | PRENSA | `10.2.92.0` | **`10.2.92.1`** | `/24` |
| **40** | MONITOREO | `10.2.93.0` | **`10.2.93.1`** | `/24` |
| **99** | NATIVA | `10.2.94.0` | **`10.2.94.1`** | `/24` |
| **50** | CONTROL | `10.2.95.0` | **`10.2.95.1`** | `/24` |

---

## 4. Enlaces WAN (Backbone)
**Máscara:** `/30` (`255.255.255.252`).
**Configuración:** Basada en Topología Final (SP conecta a Recife).

| Enlace | Router (DCE) | Puerto | IP | Router (DTE) | Puerto | IP |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Río -> SP** | Río | `Se0/3/0` | **`.100.1`** | SP | `Se0/3/0` | **`.100.2`** |
| **Río -> Sal** | Río | `Se0/2/0` | **`.100.5`** | Sal | `Se0/2/0` | **`.100.6`** |
| **Río -> Rec** | Río | `Se0/2/1` | **`.100.9`** | Rec | `Se0/2/1` | **`.100.10`**|
| **SP -> Rec** | SP | `Se0/3/1` | **`.100.13`**| Rec | `Se0/2/0` | **`.100.14`**|
| **Sal -> Rec** | Sal | `Se0/2/1` | **`.100.17`**| Rec | `Se0/3/0` | **`.100.18`**|

*Todas las IPs WAN son de la red `10.2.100.0`.*

---

## 5. Servicios Centralizados (En Río)
El Servidor Web ahora se aloja en la nueva **VLAN 50 (Control)**.

| Dispositivo | IP Estática | Gateway | VLAN |
| :--- | :--- | :--- | :--- |
| **Web Server / IoT** | **`10.2.70.10`** | `10.2.70.1` | 50 (CONTROL) |